In [ ]:
import os
import sys
import matplotlib.pyplot as plt
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
#%matplotlib qt

from data import result_loader, plotter, plotter_evaluation, plotter_layer
from data.plotter import PlotType as PT

# Experiment evaluation
This interactive notebook loads specs, histories and models from files generated by a past IMP- or OSP-experiment and plots the results.

The next cell prints all specs-files from the subdirectory `/results` and asks for the experiment to load.
Simply copy a complete line into the text field and press enter to perform the evaluation.

In [ ]:
relative_paths = result_loader.get_relative_spec_file_paths()
sep = '\n'
message = f"{sep.join(relative_paths)}\n\nPlease choose a specs-file: "
specs_file = input(message)
experiment_prefix = result_loader.extract_experiment_path_prefix(specs_file)
absolute_specs_path = result_loader.generate_absolute_specs_path(specs_file)

In [ ]:
# load experiment's specs and results
specs = result_loader.get_specs_from_file(absolute_specs_path, as_dict=False)
hists = result_loader.get_experiment_histories_from_file(experiment_prefix)
nets = result_loader.get_models_from_files(experiment_prefix, specs)

In [ ]:
specs

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,5))
plotter.plot_average_hists_on_ax(ax[0], hists.val_acc[:,::2,:120], hists.sparsity[::2], specs.plot_step, PT.VAL_ACC)
plotter.plot_average_hists_on_ax(ax[1], hists.train_loss[:,::2,:120], hists.sparsity[::2], specs.plot_step, PT.TRAIN_LOSS, force_zero=True)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,5))
plotter.plot_average_hists_on_ax(ax[0], hists.train_loss, hists.sparsity, specs.plot_step, PT.TRAIN_LOSS, force_zero=True)
plotter.plot_average_hists_on_ax(ax[1], hists.val_loss, hists.sparsity, specs.plot_step, PT.VAL_LOSS, force_zero=True)

In [ ]:
fig, ax = plt.subplots()
plotter.plot_acc_at_early_stop_on_ax(ax, hists.val_loss, hists.test_acc, hists.sparsity, specs.net, PT.TEST_ACC)

In [ ]:
fig, ax = plt.subplots()
plotter.plot_early_stop_iterations_on_ax(ax, hists.val_loss, hists.sparsity, specs.plot_step, specs.net)

In [ ]:
fig, ax = plt.subplots()
plotter.plot_average_hists_on_ax(ax, hists.test_acc[:,::2,:120], hists.sparsity[::2], specs.plot_step, PT.TEST_ACC)

In [ ]:
# compare final test-accuracies
for n in range(specs.net_count):
    unpruned_final = hists.test_acc[n, 0, -1]
    for p in range(1, specs.prune_count + 1):
        pruned_final = hists.test_acc[n, p, -1]
        print(f"Test-accuracy net #{n} (sparsity: {hists.sparsity[p]:.4}): {pruned_final:1.4} (changed by {(pruned_final-unpruned_final):1.4})")

In [ ]:
# compare final validation-accuracies
for n in range(specs.net_count):
    unpruned_final = hists.val_acc[n, 0, -1]
    for p in range(1, specs.prune_count + 1):
        pruned_final = hists.val_acc[n, p, -1]
        print(f"Validation-accuracy net #{n} (sparsity: {hists.sparsity[p]:.4}): {pruned_final:1.4} (changed by {(pruned_final-unpruned_final):1.4})")

In [ ]:
# compare test-accuracies at early-stop iteration
early_stop_indices = plotter_evaluation.find_early_stop_indices(hists.val_loss)
early_stop_acc = plotter_evaluation.get_values_at_stop_iteration(early_stop_indices, hists.test_acc)
early_stop_acc

In [ ]:
nets[0].sparsity_report()

## Evaluation with randomly reinitialized networks
Once randomly reinitialized nets have been generated and trained, they can be plotted as dotted lines to compare winning tickets and arbitrary sparse nets.
To retrain those random nets, it is necessary to generate `EarlyStopHistory` objects during the IMP-experiment.

In [ ]:
if specs.save_early_stop:
    rhists = result_loader.get_all_random_experiment_histories_from_files(experiment_prefix, specs.net_count)

In [ ]:
fig, ax = plt.subplots()
plotter.plot_average_hists_on_ax(ax, hists.test_acc[:,::,:120], hists.sparsity[::], specs.plot_step, PT.TEST_ACC, rhists.test_acc[:,::,:120])

In [ ]:
fig, ax = plt.subplots()
plotter.plot_acc_at_early_stop_on_ax(ax, hists.val_loss, hists.test_acc, hists.sparsity, specs.net, PT.TEST_ACC, rhists.val_loss, rhists.test_acc)

In [ ]:
fig, ax = plt.subplots()
plotter.plot_early_stop_iterations_on_ax(ax, hists.val_loss, hists.sparsity, specs.plot_step, specs.net, rhists.val_loss)

# example to save a figure as PDF-file
# fig.savefig("Early_stop_test.pdf", bbox_inches='tight')

## Visualization of linear and convolutional layers
The weights from linear layers and kernels from convolutional layers can be plotted.
In this example all compatible layers from the first net are shown.

In [ ]:
plotter_layer.plot_conv(nets[0].conv)

In [ ]:
plotter_layer.plot_fc(nets[0].fc)